In [ ]:
%load_ext autoreload
%autoreload 2

import json
import sys
from pathlib import Path

import numpy as np
from nb_utils.archilyse_auto_utils import plot_predictions
from nb_utils.archilyse_preds_building_prot import (
    extract_building_from_path,
    extract_site_from_path,
    process_polygon,
)
from shapely.geometry import MultiPolygon, Polygon
from tqdm import tqdm

from ifc_dl.utils.io import load_pdf_ecolution

BASE_FOLDER = Path("/ifc_dl")

archylise_path = BASE_FOLDER / "submodules/ArchilyseAuto"
sys.path.append(str(archylise_path))

from predictors.predictors.floorplans import FloorplanPredictor
from predictors.predictors.icons import IconPredictor

# from predictors.predictors.roi import RoiPredictor
from predictors.predictors.spaces import SpacePredictor
from predictors.predictors.walls import WallPredictor

## Define the pdf to predict from in this cell

In [ ]:

pdf_path = BASE_FOLDER / "data/building_prototypes/hohbächli_6_5079_zeihen/annotated/zeihen_annotated.pdf"
page_names = ['-1', '0', '1', 'section_a_b', 'sw_facade', 'nw_facade']  # by hand
idx_to_consider = [0, 1, 2]

pages_list = load_pdf_ecolution(pdf_path)
pages_with_names = [(page_names[idx], pages_list[idx]) for idx in idx_to_consider]

In [ ]:
%%capture
resources_path = archylise_path / "resources"

wall_predictor = WallPredictor(str(resources_path / "walls_model_latest.pth"))
space_predictor = SpacePredictor(str(resources_path / "spaces_model_final.pth"))

icon_pred_version = 2  # 1 or 2

version_str = "" if icon_pred_version == 1 else "_2"
icon_weights_path = str(resources_path / ("icons_model_final" + version_str)) + ".pth"
icon_predictor = IconPredictor(icon_pred_version, icon_weights_path)

In [ ]:
# choose the predictors to use to get the predictions
predictors = {
    "space": space_predictor,
    "icons": icon_predictor,
    "walls": wall_predictor,
}

# main loop for getting the predictions and saving them in a json file
final_json = []
for storey_name, page_dict in tqdm(pages_with_names):

    img = page_dict['original_img']
    page_info = page_dict['page_info']
    px_per_m = np.mean([page_info['px_m_ratio_width'], page_info['px_m_ratio_height']])

    img_dict = {}

    # get the path from "building_prototypes"
    starting_idx_path = pdf_path.parts.index("data")
    rel_path = "/".join(pdf_path.parts[starting_idx_path:])

    img_dict = {
        "img_path": rel_path,
        "coords": page_info['coords'],
        "site": extract_site_from_path(pdf_path),
        "building": extract_building_from_path(pdf_path),
        "storey": storey_name,
    }

    for predictor_name, predictor in predictors.items():
        labels, polygons, confidences = FloorplanPredictor.predict(
            predictor,
            img,
            roi=[
                (0, 0, img.shape[1], img.shape[0])
            ],  # roi is the whole image for the moment
            pixels_per_meter=px_per_m,
        )

        labels_name_json, labels_value_json, polygons_json, confidences_json = (
            [],
            [],
            [],
            [],
        )

        for label, poly, confidence in zip(labels, polygons, confidences):
            if isinstance(poly, (MultiPolygon, Polygon)):
                geometries = poly.geoms if isinstance(poly, MultiPolygon) else [poly]
                for geom in geometries:
                    labels_name_json.append(label.name)
                    labels_value_json.append(label.value)
                    polygons_json.append(process_polygon(geom))
                    confidences_json.append(confidence)
            else:
                raise ValueError("Geometry is neither Polygon nor MultiPolygon.")

        # add the predictions for this predictor
        img_dict[predictor_name] = {
            "label_values": labels_value_json,
            "label_names": labels_name_json,
            "polygons": polygons_json,
            "confidences": confidences_json,
        }

        plot_predictions(img, (labels, polygons, confidences), storey_name + ' : ' + predictor_name)

    final_json.append(img_dict)

    # save the predictions in a json file
    save_folder = Path('/ifc_dl/output') / (extract_site_from_path(pdf_path) + '_predictions')
    save_folder.mkdir(parents=True, exist_ok=True)

    with open(str(save_folder / "sample_pred.json"), "w") as f:
        json.dump(final_json, f, indent=3)
        print("The predictions have been saved!")